In [1]:
from __future__ import print_function

from sklearn.preprocessing import OneHotEncoder
from keras.layers.core import Dense, Activation, Dropout
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb
import pandas as pd
import numpy as np
import os
import re

In [2]:
def containsspecial(s):
    regex = re.compile('[@_!#$%^&*()<>?/\|}{~:]')
          
    if(regex.search(s) == None):
        return True
          
    else:
        return False
      
    

In [3]:
#parameters
maxlen = 30
labels = 2

In [57]:
cols_csv=['Sno','Name','Father_name','Husband_name','Mother_name','House_number','Gender','Age']

# cols_csv=['Sno','Name','Gender' ]

In [58]:
# input = pd.read_csv("/Users/jalend15/Downloads/cbse_gender_names_list.csv" )
input = pd.read_csv("/Users/jalend15/Downloads/combine/combined.csv" )

/Users/jalend15/opt/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3155: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [59]:
input.columns = cols_csv
print(input.head())
print(input.shape)

   Sno          Name  Father_name Husband_name Mother_name House_number  \
0  0.0   TABING TABO  TAMANG TABO          NaN         NaN          E-1   
1  1.0   TALUNG TABO  TAMANG TABO          NaN         NaN          E-1   
2  2.0    YABUR TABO  TAMANG TABO          NaN         NaN          E-1   
3  3.0  YASONG TABOH          NaN   TATE TABOH         NaN          E-1   
4  4.0   YAKEN TABOH   TATE TABOH          NaN         NaN          E-1   

   Gender Age  
0    MALE  42  
1    MALE  39  
2  FEMALE  37  
3  FEMALE  55  
4  FEMALE  34  
(21163654, 8)


In [60]:
input = input.drop_duplicates()

In [61]:
print(input.shape)

(21163648, 8)


In [74]:
# input = pd.read_csv("gender_data.csv",header=None)
# input.columns = ['name','m_or_f']
input['namelen']= [len(str(i)) for i in input['Name']]
input['sp']= [containsspecial(str(i)) for i in input['Name']]
input1 = input[(input['namelen'] >= 2) ]
input1= input1[(input1['sp']==True)]

In [75]:
input1['first_name'] = [str(name).split(' ')[0]  for name in input1['Name'].values.astype('str')]
input1['last_name'] = [' '.join(str(name).split(" ")[1:]) for name in input1['Name'].values.astype('str')]

In [76]:
# input1 = input[(input['namelen']>=2)]

In [77]:
first_names = pd.DataFrame(input1['first_name'])
print(first_names.shape)

(20867689, 1)


In [78]:
first_names = first_names.drop_duplicates()

In [79]:
msk = np.random.rand(len(first_names)) < 1
train_first_names = first_names[msk]
test_first_names = first_names[~msk]

In [80]:
print(train_first_names.shape)
print(test_first_names.shape)

(1500768, 1)
(0, 1)


In [81]:
print(train_first_names.head())

  first_name
0     TABING
1     TALUNG
2      YABUR
3     YASONG
4      YAKEN


In [82]:
input1['first_name'][0] in train_first_names['first_name'].tolist()

True

In [83]:
msk = np.ones(len(input1), dtype=bool)
print(msk.shape)

(20867689,)


In [84]:
print(input1)
input2= input1

input1 = input2.drop_duplicates().reset_index(drop=True)

               Sno                    Name    Father_name Husband_name  \
0              0.0             TABING TABO    TAMANG TABO          NaN   
1              1.0             TALUNG TABO    TAMANG TABO          NaN   
2              2.0              YABUR TABO    TAMANG TABO          NaN   
3              3.0            YASONG TABOH            NaN   TATE TABOH   
4              4.0             YAKEN TABOH     TATE TABOH          NaN   
...            ...                     ...            ...          ...   
21163647  398929.0            SIMON TAMANG     BHAKTA BDR          NaN   
21163648  398930.0         ASHRITA BASNETT  PAWAN BASNETT          NaN   
21163649  398931.0        RAJ KUMAR LIMBOO            NaN          NaN   
21163650  398932.0            ELINA RAI re            NaN          NaN   
21163653  398935.0  Husbands  - JUSEIN RAL            NaN          NaN   

         Mother_name       House_number  Gender Age  namelen    sp first_name  \
0                NaN          

In [88]:
# for i in range(len(msk)):
#   print(i)
#   if input1['first_name'][i] in train_first_names['first_name'].tolist():
#     msk[i] = True
#   else :
#     msk[i] = False


In [89]:
msk = np.random.rand(len(input1)) < 0.7

train = input1[msk]
test = input1[~msk]

In [90]:
print(len(train))

14610390


In [91]:
# train=train.sample(700000)
# test = test.sample(300000)


In [92]:
test['Gender'].value_counts()

MALE      3287641
FEMALE    2918143
Name: Gender, dtype: int64

In [93]:
print(train)
train = train.drop_duplicates().reset_index(drop=True)
test = test.drop_duplicates().reset_index(drop=True)

               Sno                    Name  Father_name Husband_name  \
0              0.0             TABING TABO  TAMANG TABO          NaN   
1              1.0             TALUNG TABO  TAMANG TABO          NaN   
2              2.0              YABUR TABO  TAMANG TABO          NaN   
4              4.0             YAKEN TABOH   TATE TABOH          NaN   
5              5.0             YAPET TABOH   TATE TABOH          NaN   
...            ...                     ...          ...          ...   
20867683  398928.0     DANIAL SHANKER KAMI  SHYAM KUMAR          NaN   
20867684  398929.0            SIMON TAMANG   BHAKTA BDR          NaN   
20867686  398931.0        RAJ KUMAR LIMBOO          NaN          NaN   
20867687  398932.0            ELINA RAI re          NaN          NaN   
20867688  398935.0  Husbands  - JUSEIN RAL          NaN          NaN   

         Mother_name       House_number  Gender Age  namelen    sp first_name  \
0                NaN                E-1    MALE  42   

In [94]:
names = input['Name'].str.lower()
gender = input['Gender']
vocab = set(' '.join([str(i) for i in names]))
vocab.add('SEP')
vocab.add('CLS')
len_vocab = len(vocab)

In [95]:
print(vocab)
print("vocab length is ",len_vocab)
print ("length of input is ",len(input1))

{'$', ';', '@', '€', '=', '2', '&', '_', 'r', '1', '»', ':', '™', '9', 'b', 't', 'g', '~', '3', 'w', '’', '#', '‘', 'CLS', '!', '8', 'o', '«', '|', '“', '£', '+', 'f', '[', '6', 'd', '{', 'SEP', '”', '0', 'q', '/', 'y', ')', 'k', 'a', '7', ' ', '¥', '©', "'", '<', 'c', '-', '"', 'e', 'z', '*', '°', 's', '%', '?', 'j', '®', 'p', '¢', '}', 'x', '—', ']', '>', '\\', 'h', '§', '.', 'v', '(', '5', 'n', ',', 'u', 'l', '4', 'i', 'm', 'é'}
vocab length is  86
length of input is  20867689


In [96]:
char_index = dict((c, i) for i, c in enumerate(vocab))

In [97]:
print(char_index)

{'$': 0, ';': 1, '@': 2, '€': 3, '=': 4, '2': 5, '&': 6, '_': 7, 'r': 8, '1': 9, '»': 10, ':': 11, '™': 12, '9': 13, 'b': 14, 't': 15, 'g': 16, '~': 17, '3': 18, 'w': 19, '’': 20, '#': 21, '‘': 22, 'CLS': 23, '!': 24, '8': 25, 'o': 26, '«': 27, '|': 28, '“': 29, '£': 30, '+': 31, 'f': 32, '[': 33, '6': 34, 'd': 35, '{': 36, 'SEP': 37, '”': 38, '0': 39, 'q': 40, '/': 41, 'y': 42, ')': 43, 'k': 44, 'a': 45, '7': 46, ' ': 47, '¥': 48, '©': 49, "'": 50, '<': 51, 'c': 52, '-': 53, '"': 54, 'e': 55, 'z': 56, '*': 57, '°': 58, 's': 59, '%': 60, '?': 61, 'j': 62, '®': 63, 'p': 64, '¢': 65, '}': 66, 'x': 67, '—': 68, ']': 69, '>': 70, '\\': 71, 'h': 72, '§': 73, '.': 74, 'v': 75, '(': 76, '5': 77, 'n': 78, ',': 79, 'u': 80, 'l': 81, '4': 82, 'i': 83, 'm': 84, 'é': 85}


In [98]:
#take input upto max and truncate rest
#encode to vector space(one hot encoding)
#padd 'END' to shorter sequences
train_X = []
train_Y=[]
train_texts= []
train_labels=[]
trunc_train_name = [str(i).lower()[0:30] for i in train.Name]
for i in trunc_train_name:
    tmp = [char_index[j] for j in str(i)]
    tmp.append(char_index["SEP"])
    tmp.insert(0,char_index["CLS"])
    train_X.append(tmp)
    train_texts.append(i)
for i in train.Gender:
    train_labels.append(i)
    if i == 'MALE':
        train_Y.append(1)
    else:
        train_Y.append(0)
train_labels =train_Y

In [99]:
np.asarray(train_X).shape

(14610390,)

In [100]:
test_X = []
test_Y = []
test_texts=[]
test_labels = []
trunc_test_name = [str(i).lower()[0:maxlen] for i in test.Name]
for i in trunc_test_name:
    tmp = [(char_index[j]) for j in str(i)]
    tmp.append((char_index["SEP"]))
    tmp.insert(0,(char_index["CLS"]))
    test_X.append(tmp)
    test_texts.append(i)
for i in test.Gender:
    test_labels.append(i)
    if i == 'MALE':
        test_Y.append(1)
    else:
        test_Y.append(0)
test_labels =test_Y

In [101]:
train_X = np.asarray(train_X)
train_Y = np.asarray(train_Y)
test_X = np.asarray(test_X)
test_Y = np.asarray(test_Y)

In [102]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=0.2)


In [103]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

loading file https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt from cache at /Users/jalend15/.cache/huggingface/transformers/0e1bbfda7f63a99bb52e3915dcf10c3c92122b827d92eb2d34ce94ee79ba486c.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/tokenizer.json from cache at /Users/jalend15/.cache/huggingface/transformers/75abb59d7a06f4f640158a9bfcde005264e59e8d566781ab1415b139d2e4c603.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/tokenizer_config.json from cache at /Users/jalend15/.cache/huggingface/transformers/8c8624b8ac8aa99c60c912161f8332de003484428c47906d7ff7eb7f73eecd

In [104]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

KeyboardInterrupt: 

In [ ]:
print(len(train_texts))

In [ ]:
import torch

class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, train_labels)
val_dataset = IMDbDataset(val_encodings, val_labels)
test_dataset = IMDbDataset(test_encodings,test_labels)




In [ ]:
print(test_dataset)

In [47]:
def compute_metrics(p):    
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1} 

from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=0.1,              # total number of training epochs
    per_device_train_batch_size=64,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics, 
   )
                                         # evaluation dataset


trainer.train()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'pre_clas

NameError: name 'val_dataset' is not defined

In [48]:
from transformers import BertConfig, BertModel
model1 = DistilBertForSequenceClassification.from_pretrained('./results_ER_Full/checkpoint-17000/')

In [ ]:
# from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# metrics=trainer.evaluate()
# print(metrics)


In [ ]:
# from torch.utils.data import DataLoader
# from transformers import DistilBertForSequenceClassification, AdamW

# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# model1 = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
# model1.to(device)
# model1.train()

# train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# optim = AdamW(model1.parameters(), lr=5e-5)

# for epoch in range(3):
#     for batch in train_loader:
#         optim.zero_grad()
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         labels = batch['labels'].to(device)
#         outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
#         loss = outputs[0]
#         loss.backward()
#         optim.step()

# model1.eval()


In [50]:
# #test_trainer=Trainer(model)
trainer = Trainer(
    model=model1,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
#     eval_dataset=val_dataset,
    compute_metrics=compute_metrics, 
   )
         

In [137]:
# model=model1

# import torch.utils.data as data_utils

# indices = torch.arange(10000)
# tr_10k = data_utils.Subset(test_dataset, indices)


In [51]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score



In [139]:
 from torch.utils.data import Dataset, DataLoader

test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
raw_pred, _, _ = trainer.prediction_loop(test_loader, description="prediction")

***** Running prediction *****
  Num examples = 300000
  Batch size = 64
/Users/jalend15/opt/miniconda3/lib/python3.8/site-packages/transformers/trainer_pt_utils.py:376: FutureWarning: DistributedTensorGatherer is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [52]:
 from torch.utils.data import Dataset, DataLoader

test_loader = DataLoader(train_dataset, batch_size=64, shuffle=False)
raw_pred, _, _ = trainer.prediction_loop(test_loader, description="prediction")

***** Running prediction *****
  Num examples = 205530
  Batch size = 64
/Users/jalend15/opt/miniconda3/lib/python3.8/site-packages/transformers/trainer_pt_utils.py:376: FutureWarning: DistributedTensorGatherer is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [53]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

def compute_metrics1(p,z):
    pred, labels = p,z
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}


In [56]:
y_pred = np.argmax(raw_pred, axis=1)
print(len(y_pred))
print(raw_pred)
#print(test_Y)
results = compute_metrics1(raw_pred,train_labels)
print(results)

205530
[[ 2.8389678 -2.4154549]
 [ 2.7212958 -2.297977 ]
 [ 3.0571337 -2.5950747]
 ...
 [-1.9216907  1.895895 ]
 [-1.2466686  1.1756608]
 [-1.2018938  1.1364172]]
{'accuracy': 0.9560842699362624, 'precision': 0.9528814319216443, 'recall': 0.9687850631136045, 'f1': 0.9607674386257736}


In [55]:
from sklearn.metrics import confusion_matrix
y_pred= np.argmax(raw_pred,axis=1)
matrix = confusion_matrix(train_labels, y_pred)
matrix.diagonal()/matrix.sum(axis=1)

array([0.94024057, 0.96878506])

In [141]:
y_pred = np.argmax(raw_pred, axis=1)
print(y_pred)
#print(test_Y)
results = compute_metrics1(raw_pred,test_Y)
print(results)

[0 0 0 ... 1 1 0]
{'accuracy': 0.9421666666666667, 'precision': 0.9437417596628795, 'recall': 0.946644055355527, 'f1': 0.9451906795723926}


In [142]:
from sklearn.metrics import confusion_matrix
y_pred= np.argmax(raw_pred,axis=1)
matrix = confusion_matrix(test_Y, y_pred)
matrix.diagonal()/matrix.sum(axis=1)

array([0.93718258, 0.94664406])